In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from keras.models import load_model
import streamlit as st
import plotly.graph_objects as go
from datetime import datetime
import pandas_ta as ta

In [2]:
df = yf.download('AAPL', '2015-01-01', '2023-01-01')
df = df.reset_index()   
df['RSI'] = ta.rsi(df.Close, length=12)
df

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,RSI
0,2015-01-02,27.847500,27.860001,26.837500,27.332500,24.531765,212818400,NaN
1,2015-01-05,27.072500,27.162500,26.352501,26.562500,23.840666,257142000,NaN
2,2015-01-06,26.635000,26.857500,26.157499,26.565001,23.842913,263188400,NaN
3,2015-01-07,26.799999,27.049999,26.674999,26.937500,24.177235,160423600,NaN
4,2015-01-08,27.307501,28.037500,27.174999,27.972500,25.106184,237458000,NaN
...,...,...,...,...,...,...,...,...
2009,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.477127,63814900,34.153693
2010,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.652435,69007800,31.848873
2011,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.674026,85438400,27.443800
2012,2022-12-29,127.989998,130.479996,127.730003,129.610001,129.233658,75703700,36.073997


In [3]:
from holidays import WEEKEND, HolidayBase
from dateutil.easter import easter
from dateutil.relativedelta import relativedelta as rd
from prophet import Prophet
import copy


#Check if NA values are in data
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df['y'] = df['Close']
#df['ds']=pd.to_datetime(df['ds'], format = '%d.%m.%Y %H:%M:%S').dt.date #for daily timeframe
df['ds']=pd.to_datetime(df['Date'])

def prophet_signal(df, l, backcandles, frontpredictions, diff_limit, signal):
    dfsplit = copy.deepcopy(df[l-backcandles:l+1])
    model=Prophet()
    model.fit(dfsplit) #the algo runs at the closing time of the current candle which is included in the fit
    future = model.make_future_dataframe(periods=frontpredictions, include_history=False)
    forecast = model.predict(future)
    if signal:
        if(forecast.yhat.mean()-dfsplit.close.iat[-1]<diff_limit):
            return 1
        elif(forecast.yhat.mean()-dfsplit.close.iat[-1]>diff_limit):
            return 2
        else:
            return 0
    else:
        forecast["y_target"] = df['y'].iloc[l+1:l+frontpredictions+1]
        forecast = forecast[['yhat', 'yhat_lower', 'yhat_upper']].values[0]
        return forecast[0],forecast[1],forecast[2] 

prophet_signal(df, 200, 100, frontpredictions=1, diff_limit=0.008, signal=False)


from tqdm import tqdm
backcandles=100
frontpredictions=1

yhatlow = [0 for i in range(len(df))]
yhat = [0 for i in range(len(df))]
yhathigh = [0 for i in range(len(df))]

for row in tqdm(range(backcandles, len(df)-frontpredictions)):
    prophet_pred = prophet_signal(df, row, backcandles, frontpredictions=1, diff_limit=0.005, signal=False)
    yhat[row] = prophet_pred[0]
    yhatlow[row] = prophet_pred[1]
    yhathigh[row] = prophet_pred[2]

df["yhat"] = yhat
df["yhatlow"] = yhatlow
df["yhathigh"] = yhathigh

df['yhatlow'] = df['yhatlow'].shift(+1)
df['yhathigh'] = df['yhathigh'].shift(+1)
df['yhat'] = df['yhat'].shift(+1)

from matplotlib import pyplot as plt
x1 = 200
x2 = 250
plt.figure(figsize=(18,6))
plt.plot(df['ds'].iloc[x1:x2],df['yhatlow'].iloc[x1:x2], label="Prediction", marker='o')
plt.fill_between(df['ds'].iloc[x1:x2], df['yhat'].iloc[x1:x2], df['yhathigh'].iloc[x1:x2], color='b', alpha=.1)
plt.plot(df['ds'].iloc[x1:x2], df['y'].iloc[x1:x2], label = "Market", marker='o')
plt.legend(loc="upper left")

15:55:25 - cmdstanpy - INFO - Chain [1] start processing
15:55:25 - cmdstanpy - INFO - Chain [1] done processing
15:55:25 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
15:55:25 - cmdstanpy - INFO - Chain [1] start processing
15:55:25 - cmdstanpy - INFO - Chain [1] done processing
15:55:25 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/prophet/stan_model/prophet_model.bin random seed=32984 data file=/var/folders/87/cxqpfc9x3vd1j0n6lhnp833w0000gn/T/tmpa8en3kh6/97atdr1d.json init=/var/folders/87/cxqpfc9x3vd1j0n6lhnp833w0000gn/T/tmpa8en3kh6/4lkfcftq.json output file=/var/folders/87/cxqpfc9x3vd1j0n6lhnp833w0000gn/T/tmpa8en3kh6/prophet_modeleisj64z0/prophet_model-20230716155525.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[4613]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <CD2A54F0-FD72-3F85-95F6-81CEA1D16EAA> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/87/cxqpfc9x3vd1j0n6lhnp833w0000gn/T/pip-install-s5152wfl/prophet_4d3a92d8b2624dabb03baf3252312c86/build/lib.macosx-10.9-universal2-cpython-311/prophet/stan_model/cmdstan-2.31.0/stan/